# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data = pd.read_csv("clean_city_data.csv")
clean_city_data



,City,lat,lng,temp,humidity,cloudiness,wind,country,date
0,Sitka,57.05,-135.33,50.00,93,90,5.82,US,1593166611
1,Nerchinskiy Zavod,51.31,119.61,71.49,44,69,5.59,RU,1593166611
2,Atuona,-9.80,-139.03,79.74,79,28,15.37,PF,1593166611
3,Laguna,38.42,-121.42,68.00,64,5,3.36,US,1593166611
4,Butler,39.45,-84.57,68.00,100,20,4.52,US,1593166611
...,...,...,...,...,...,...,...,...,...
543,Melito di Porto Salvo,37.92,15.77,84.00,54,5,5.82,IT,1593166653
544,San Rafael,-34.62,-68.33,40.44,61,95,4.50,AR,1593166653
545,Brigantine,39.41,-74.36,70.00,88,40,6.20,US,1593166653
546,Baní,18.28,-70.33,79.00,88,44,3.00,DO,1593166653


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

clean_city_lat_lng = clean_city_data[["lat","lng"]]

humidity = clean_city_data["humidity"]

#plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(clean_city_lat_lng, weights = humidity, dissipating=False, max_intensity = 10, point_radius =1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_data = clean_city_data.loc[(clean_city_data["temp"]<80) & (clean_city_data["temp"]>70)&(clean_city_data["wind"]<10)&(clean_city_data["cloudiness"]==0)]
hotel_df= weather_data.dropna(how='any')
hotel_df

,City,lat,lng,temp,humidity,cloudiness,wind,country,date
7,Morondava,-20.28,44.28,78.76,61,0,7.29,MG,1593166611
38,Sobolevo,54.43,31.90,78.31,53,0,4.25,RU,1593166613
188,Cabo San Lucas,22.89,-109.91,72.00,87,0,1.99,MX,1593166625
190,Chioggia,45.23,12.29,79.00,60,0,5.82,IT,1593166625
196,Chokwé,-24.53,32.98,77.76,41,0,5.73,MZ,1593166625
233,Zhigansk,66.77,123.37,78.22,36,0,5.26,RU,1593166628
256,Karratha,-20.74,116.85,71.35,40,0,6.80,AU,1593166630
506,Chibuto,-24.69,33.53,74.91,49,0,4.81,MZ,1593166650


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'radius': 5000,
          'type': 'lodging',
          'key':g_key}


for index, row in hotel_df.iterrows():
    try:
        params['location'] = f"{row['lat']},{row['lng']}"
        city = row['City']
        
        response = requests.get(base_url, params=params)
        hotel_json = response.json()
        hotel_name = (hotel_json['results'][0]['name'])
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
        print(f"{city} Nearest hotel is {hotel_name}" )

    except (KeyError, IndexError):
        print(f"{city} not found. Skipping...")

Morondava Nearest hotel is La Case Bambou
Sobolevo not found. Skipping...
Cabo San Lucas Nearest hotel is Hotel Tesoro Los Cabos
Chioggia Nearest hotel is Hotel Grande Italia 4S
Chokwé Nearest hotel is Complexo Sonho Real
Zhigansk Nearest hotel is Gostinichnyy Kompleks Aylgy
Karratha Nearest hotel is ibis Styles Karratha
Chibuto Nearest hotel is Hotel Mar Da Zinha


In [6]:
hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,lat,lng,temp,humidity,cloudiness,wind,country,date,Hotel Name
7,Morondava,-20.28,44.28,78.76,61,0,7.29,MG,1593166611,La Case Bambou
188,Cabo San Lucas,22.89,-109.91,72.00,87,0,1.99,MX,1593166625,Hotel Tesoro Los Cabos
190,Chioggia,45.23,12.29,79.00,60,0,5.82,IT,1593166625,Hotel Grande Italia 4S
196,Chokwé,-24.53,32.98,77.76,41,0,5.73,MZ,1593166625,Complexo Sonho Real
233,Zhigansk,66.77,123.37,78.22,36,0,5.26,RU,1593166628,Gostinichnyy Kompleks Aylgy
256,Karratha,-20.74,116.85,71.35,40,0,6.80,AU,1593166630,ibis Styles Karratha
506,Chibuto,-24.69,33.53,74.91,49,0,4.81,MZ,1593166650,Hotel Mar Da Zinha


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# fig.add_layer(marker_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))